In [ ]:
""""
We use unstrumental variables approach to insert previous values of complexity index 
to the right-hand side of uor final equation
"""

# Plan
# 1. Build an autoregressive model of eci index versions
# 2. Predicting values of eci index version
# 3. Adding them to the dataset in form applicable for the final model estimation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir("C:\\Users\\Vladimir.Grigoryev\\Documents\\Python Scripts\\eci")

In [71]:
path = pd.read_csv("5y_dataset_transformed.csv")
dd = pd.DataFrame(path)
dd.drop("Unnamed: 0", axis = 1, inplace=True)
dd.head()

,eci1,eci2,eci3,gdp_pc,population,res_exp1_to_gdp,res_exp2_to_gdp,res_exp3_to_gdp,oil_rents,res_rents,oil_prod_pc,map,code,year,T,t
0,NaN,NaN,NaN,NaN,15985.182581,NaN,NaN,NaN,NaN,NaN,NaN,ABW1960,ABW,1960,12,1
1,NaN,NaN,NaN,NaN,16280.982634,NaN,NaN,NaN,NaN,NaN,NaN,ABW1965,ABW,1965,12,2
2,NaN,NaN,NaN,NaN,15863.412421,NaN,NaN,NaN,NaN,NaN,NaN,ABW1970,ABW,1970,12,3
3,NaN,NaN,NaN,NaN,13291.931626,NaN,NaN,NaN,NaN,NaN,NaN,ABW1975,ABW,1975,12,4
4,NaN,NaN,NaN,NaN,9668.204912,NaN,NaN,NaN,NaN,NaN,NaN,ABW1980,ABW,1980,12,5


In [4]:
# 1. Building autoregressive model

# Choosing number of lags using ACF and PACF
# Looking inside the python script "What lags for ECI. ACF and PACF" reveals that the optimal number of lags 
# for most of the countries is 1.

from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error

In [103]:
# 2. Creating predicted eci1 values
countries = list(dd.code.unique())
predicted = []
actual = []
previous = []
for name in countries:
    filt = dd["code"] == name
    series = pd.Series(dd[filt]["eci1"].dropna())
    if len(series) > 1:
        X = series.values
        model = AR(X)
        model_fit = model.fit(maxlag=1)
        X_hat = model_fit.predict()
        actual.extend(X)
        predicted.extend([np.nan, np.nan])
        predicted.extend(X_hat)
        predicted.pop(-1)
        previous.extend([np.nan])
        previous.extend(X)
        previous.pop(-1)

joint1 = pd.DataFrame([actual, predicted, previous]).T
joint1.columns = ["eci1", "eci1_hat_t1", "eci1_t1"]
joint1.head()

,eci1,eci1_hat_t1,eci1_t1
0,-0.844010,NaN,NaN
1,0.160685,NaN,-0.844010
2,0.295374,0.223426,0.160685
3,-0.120289,0.133552,0.295374
4,-0.096062,0.121504,-0.120289


In [106]:
# 2. Creating predicted eci2 values
countries = list(dd.code.unique())
predicted = []
actual = []
previous = []
for name in countries:
    filt = dd["code"] == name
    series = pd.Series(dd[filt]["eci2"].dropna())
    if len(series) > 1:
        X = series.values
        model = AR(X)
        model_fit = model.fit(maxlag=1)
        X_hat = model_fit.predict()
        actual.extend(X)
        predicted.extend([np.nan, np.nan])
        predicted.extend(X_hat)
        predicted.pop(-1)
        previous.extend([np.nan])
        previous.extend(X)
        previous.pop(-1)

joint2 = pd.DataFrame([actual, predicted, previous]).T
joint2.columns = ["eci2", "eci2_hat_t1", "eci2_t2"]
joint2.head()

,eci2,eci2_hat_t1,eci2_t2
0,-0.788619,NaN,NaN
1,0.222153,NaN,-0.788619
2,0.166358,0.261046,0.222153
3,-0.115194,0.118310,0.166358
4,-0.059322,0.126189,-0.115194


In [107]:
# 2. Creating predicted eci3 values
countries = list(dd.code.unique())
predicted = []
actual = []
previous = []
for name in countries:
    filt = dd["code"] == name
    series = pd.Series(dd[filt]["eci3"].dropna())
    if len(series) > 1:
        X = series.values
        model = AR(X)
        model_fit = model.fit(maxlag=1)
        X_hat = model_fit.predict()
        actual.extend(X)
        predicted.extend([np.nan, np.nan])
        predicted.extend(X_hat)
        predicted.pop(-1)
        previous.extend([np.nan])
        previous.extend(X)
        previous.pop(-1)

joint3 = pd.DataFrame([actual, predicted, previous]).T
joint3.columns = ["eci3", "eci3_hat_t1", "eci3_t1"]
joint3.head()

,eci3,eci3_hat_t1,eci3_t1
0,-0.816777,NaN,NaN
1,0.326727,NaN,-0.816777
2,0.072476,0.373223,0.326727
3,-0.070814,0.058437,0.072476
4,-0.094526,0.128427,-0.070814


In [74]:
# checking whether I can merge data using actual data
len(joint), joint.eci1.nunique(), joint.eci1_hat.nunique(), joint.eci1_hat.isna().sum()
# every value of eci1 is unique -> yes, we can merge

(2249, 2249, 2005, 244)

In [108]:
# 3. Adding them to the dataset in form applicable for the final model estimation
gh = pd.merge(dd, joint1, how="outer", on="eci1")
gh = pd.merge(gh, joint2, how="outer", on="eci2")
gh = pd.merge(gh, joint3, how="outer", on="eci3")
gh.tail(10)

,eci1,eci2,eci3,gdp_pc,population,res_exp1_to_gdp,res_exp2_to_gdp,res_exp3_to_gdp,oil_rents,res_rents,...,code,year,T,t,eci1_hat_t1,eci1_t1,eci2_hat_t1,eci2_t2,eci3_hat_t1,eci3_t1
3442,0.603522,0.521368,0.507053,265.236663,4.078156e+06,0.000088,0.015190,0.046197,NaN,NaN,...,ZWE,1965,12,2,NaN,NaN,NaN,NaN,NaN,NaN
3443,0.747302,0.760474,0.958551,427.813724,4.731958e+06,0.000007,0.005561,0.009501,0.0,2.593689,...,ZWE,1970,12,3,NaN,0.603522,NaN,0.521368,NaN,0.507053
3444,0.353467,0.235512,0.389652,581.237349,5.455811e+06,0.000004,0.006570,0.009773,0.0,3.663159,...,ZWE,1975,12,4,0.321520,0.747302,0.191949,0.760474,0.232408,0.958551
3445,-0.304678,-0.597882,-0.562726,793.082518,6.333032e+06,0.000101,0.027742,0.059559,0.0,3.515957,...,ZWE,1980,12,5,0.423536,0.353467,0.336777,0.235512,0.458139,0.389652
3446,-0.360760,-0.549257,-0.549708,590.338188,7.369822e+06,0.001381,0.045303,0.094463,0.0,3.376052,...,ZWE,1985,12,6,0.144099,-0.304678,0.018803,-0.597882,0.173712,-0.562726
3447,-0.222668,-0.314491,-0.068154,539.166849,8.114164e+06,0.000324,0.041831,0.117836,0.0,6.202805,...,ZWE,1990,12,7,-0.322876,-0.360760,-0.485991,-0.549257,-0.302439,-0.549708
3448,-0.296282,-0.291387,0.173812,458.789235,8.372756e+06,0.000959,0.039742,0.129421,0.0,4.287835,...,ZWE,1995,12,8,-0.362668,-0.222668,-0.456539,-0.314491,-0.295930,-0.068154
3449,-0.379419,-0.287096,0.038624,326.120585,8.119033e+06,0.000558,0.054733,0.137620,0.0,3.626012,...,ZWE,2000,12,9,-0.264687,-0.296282,-0.314339,-0.291387,-0.055173,0.173812
3450,-0.287597,-0.160549,0.068513,248.445299,7.268265e+06,0.000362,0.152875,0.206345,0.0,7.082205,...,ZWE,2005,12,10,-0.316918,-0.379419,-0.300344,-0.287096,0.065801,0.038624
3451,-0.314022,-0.323848,-0.160679,391.142171,5.207840e+06,0.000273,0.017767,0.040283,0.0,2.744429,...,ZWE,2010,12,11,-0.375907,-0.287597,-0.297745,-0.160549,-0.001788,0.068513


In [109]:
gh.to_csv("5y_dataset_transformed_iv.csv")

In [ ]:
# END